In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import malaya

/home/ubuntu/.local/lib/python3.8/site-packages/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3361
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/ubuntu/.local/lib/python3.8/site-packages/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3879
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [3]:
model = malaya.tatabahasa.transformer_tagging(model = 'small')

Downloading:   0%|          | 0.00/397M [00:00<?, ?B/s]

2022-11-20 11:50:56.251428: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-20 11:50:56.263811: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-20 11:50:56.263837: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: huseincomel-desktop
2022-11-20 11:50:56.263840: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: huseincomel-desktop
2022-11-20 11:50:56.263883: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.141.3
2022-11-20 11:50:56.263896: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] ke

In [4]:
import json

data = []
with open('test.jsonl') as fopen:
    for l in fopen:
        data.append(json.loads(l))

In [5]:
import collections

def compute_exact(a_gold, a_pred):
    return int(a_gold == a_pred)


def compute_f1(a_gold, a_pred):
    gold_toks = a_gold
    pred_toks = a_pred
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

In [6]:
o = model.greedy_decoder(['saya tak suka'])[0]
o

[('Dia', 11), ('tak', 2), ('suka', 2)]

In [7]:
p_y = [o_[0] for o_ in o]
p_t = [o_[1] for o_ in o]
p_y

['Dia', 'tak', 'suka']

In [ ]:
from tqdm import tqdm

exact_match, f1, exact_match_tag, f1_tag = [], [], [], []
for i in tqdm(range(len(data))):
    x = ' '.join([d[0] for d in data[i][1]])
    y = [d[0] for d in data[i][0]]
    tag = [d[1] for d in data[i][1]]
    
    o = model.greedy_decoder([x])[0]
    
    p_y = [o_[0] for o_ in o]
    p_t = [o_[1] for o_ in o]
    
    exact_match.append(compute_exact(y, p_y))
    exact_match_tag.append(compute_exact(tag, p_t))
    f1.append(compute_f1(y, p_y))
    f1_tag.append(compute_f1(tag, p_t))

 70%|████████████████████████████████████████████████████████████████▋                           | 3514/4994 [10:44<04:55,  5.00it/s]

In [9]:
import numpy as np

In [10]:
np.mean(exact_match), np.mean(exact_match_tag)

(0.7891067538126362, 0.8836601307189542)

In [11]:
np.mean(f1), np.mean(f1_tag)

(0.9678974144774667, 0.9831785594259043)